In [3]:
import subprocess, os, json

VIDEO_ROOT = "videos/"
VIDEO_FRAMES_ROOT = "video-frames/"

def get_immediate_subdirectories(a_dir):
    return [name for name in os.listdir(a_dir)
            if os.path.isdir(os.path.join(a_dir, name))]

def many_jsons_to_one_json(analysis_path):
    # many jsons to one json

    def get_frame_from_fn(fn_hi): 
        k = fn_hi.split("_")[0].split("/")[-1]
        return int(k)

    face_tracking_dir = analysis_path
    fns = [face_tracking_dir + f for f in os.listdir(face_tracking_dir) if "landmarks" in f]
    out_json = {}

    for fn in fns:

        with open(fn) as f: 
            j = json.load(f)
            res = j["result"]

            frame_number = str(get_frame_from_fn(fn))
            out_json[frame_number] = []

            for result in res:
                out_json[frame_number].append({
                        "face_minBBox": result["face_minBBox"],
                        "face_maxBBox": result["face_maxBBox"]
                    })
    with open(face_tracking_dir + "out.json", 'w') as f_out:
        json.dump(out_json, f_out)

In [2]:
many_jsons_to_one_json("analysis/face-tracking/equation/")

In [4]:
all_videos = [v for v in os.listdir(VIDEO_ROOT) if "small" not in v and v[0] !='.' if "arctic" not in v and "congo" not in v and "equation" not in v]

for video in all_videos:
    base = video.lower().split(".mp4")[0]
    video_path = VIDEO_ROOT + video
    video_frames_path = VIDEO_FRAMES_ROOT + base + "/"
    
    # delete any existing video frames
    sub_directory_paths = [VIDEO_FRAMES_ROOT + p + "/" for p in get_immediate_subdirectories(VIDEO_FRAMES_ROOT)]
    for d in sub_directory_paths: 
        for frame in os.listdir(d):
            os.remove(d + frame)
            
    # make a new directory
    if not os.path.exists(video_frames_path):
        os.mkdir(video_frames_path)
        
    analysis_path = "analysis/face-tracking/" + base + "/"
    if not os.path.exists(analysis_path): 
        os.mkdir(analysis_path)
        
        # split the video into frames
        # subprocess.call(["ffmpeg", "-i", video_path, video_frames_path + "%05d.png"])
        subprocess.call(["ffmpeg", "-i", video_path, "-vf", "fps=2", video_frames_path + "%05d.png"])

        # run facetracking analysis
        subprocess.call(["../../../../../resources/OpenFace/bin/FaceLandmarkImg", '-q', '-multi-view', '1',
                         '-wild', '-gaze', '-fdir', video_frames_path, "-ofdir", analysis_path])
        # pts to json
        subprocess.call(["node", "analysis/of2json.js", analysis_path])

        # many jsons to one json
        many_jsons_to_one_json(analysis_path)

In [11]:
out_json.keys()

['344',
 '345',
 '346',
 '347',
 '340',
 '341',
 '342',
 '343',
 '348',
 '349',
 '298',
 '299',
 '296',
 '297',
 '294',
 '295',
 '292',
 '293',
 '290',
 '291',
 '270',
 '271',
 '272',
 '273',
 '274',
 '275',
 '276',
 '277',
 '278',
 '279',
 '108',
 '109',
 '102',
 '103',
 '100',
 '101',
 '106',
 '107',
 '104',
 '105',
 '1001',
 '1210',
 '99',
 '98',
 '91',
 '90',
 '93',
 '92',
 '95',
 '94',
 '97',
 '96',
 '559',
 '558',
 '555',
 '554',
 '557',
 '556',
 '551',
 '550',
 '553',
 '552',
 '1439',
 '1199',
 '1198',
 '1191',
 '1190',
 '1193',
 '1192',
 '1195',
 '1194',
 '1197',
 '1196',
 '1177',
 '1176',
 '1175',
 '1174',
 '1173',
 '1172',
 '1171',
 '1170',
 '1179',
 '1178',
 '511',
 '510',
 '513',
 '1285',
 '1284',
 '1287',
 '512',
 '1281',
 '1280',
 '1283',
 '1282',
 '1003',
 '879',
 '1289',
 '1288',
 '514',
 '689',
 '688',
 '685',
 '684',
 '687',
 '686',
 '681',
 '680',
 '683',
 '682',
 '458',
 '1226',
 '621',
 '873',
 '1224',
 '1223',
 '1222',
 '1221',
 '1220',
 '407',
 '406',
 '405',
 '1

In [21]:
"bin/FaceLandmarkImg -q -multi-view 1 -wild -gaze -fdir ../../Sites/360-video-project/vrview/examples/orientations/video-frames/nocuts/ -ofdir ../../Sites/360-video-project/vrview/examples/orientations/analysis/face-tracking/nocuts/".split()

['bin/FaceLandmarkImg',
 '-q',
 '-multi-view',
 '1',
 '-wild',
 '-gaze',
 '-fdir',
 '../../Sites/360-video-project/vrview/examples/orientations/video-frames/nocuts/',
 '-ofdir',
 '../../Sites/360-video-project/vrview/examples/orientations/analysis/face-tracking/nocuts/']

In [ ]:
many